In [199]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
import sqlite3
import ast
import random
print(__version__) # requires version >= 1.9.0

3.4.1


In [200]:
init_notebook_mode(connected=True)


In [201]:
# sqlite_file = "../itsim/itsim/datastore/storage/sqlite_01.sqlite"
sqlite_file = "data/sqlite_01.sqlite"
conn = sqlite3.connect(sqlite_file)

In [202]:
node_uuid_idx = 0
node_timestamp_idx = 1
node_simulation_idx = 2
node_json_idx = 3

df = []

with conn:

    cursor = conn.cursor()
    cursor.execute('SELECT * FROM network_event')
    all_entries = cursor.fetchall()
    
    # DEBUG
    print(all_entries)
    
    cnt = 0
    colors = []

    net_events_post_proc = []

    for network_event in all_entries:
        data = ast.literal_eval(network_event[node_json_idx])      
        uuid_node = data["uuid_node"]
        event_info = {"uuid_node": uuid_node,"start":"","stop":""}
        net_events_post_proc.append(event_info)

    for network_event in all_entries:
        data = ast.literal_eval(network_event[node_json_idx])      
        uuid_node = data["uuid_node"]   
        event_dict = next(item for item in net_events_post_proc if item["uuid_node"] == uuid_node)
        if event_dict["start"]=="":
            event_dict["start"]=network_event[node_timestamp_idx]
        else:
            event_dict["stop"]=network_event[node_timestamp_idx]
    
    #print(net_events_post_proc)
           

[('c7adc825-835d-4137-89ba-150d3fb19958', '2018-11-20T13:14:36.010016', '9cca45d3-df3f-45cc-9f71-c99f3b9b1735', '{"sim_uuid": "9cca45d3-df3f-45cc-9f71-c99f3b9b1735", "timestamp": "2018-11-20T13:14:36.010016", "type": "network_event", "uuid": "c7adc825-835d-4137-89ba-150d3fb19958", "network_event_type": "open", "uuid_node": "e77e5e50-e88c-46e1-8483-cfe731f2a1dc", "protocol": "UDP", "pid": 0, "src": ["", 68], "dst": ["0.0.0.0", 0]}'), ('6760c4a0-c9cb-4e49-89c1-524b62988cd8', '2018-11-20T13:14:36.010016', '9cca45d3-df3f-45cc-9f71-c99f3b9b1735', '{"sim_uuid": "9cca45d3-df3f-45cc-9f71-c99f3b9b1735", "timestamp": "2018-11-20T13:14:36.010016", "type": "network_event", "uuid": "6760c4a0-c9cb-4e49-89c1-524b62988cd8", "network_event_type": "open", "uuid_node": "96bf3153-01bc-4115-84a5-c0faea61ff82", "protocol": "UDP", "pid": 0, "src": ["", 68], "dst": ["0.0.0.0", 0]}'), ('fbce39cd-5455-45da-9b74-3fd72dd010a9', '2018-11-20T13:14:36.010016', '9cca45d3-df3f-45cc-9f71-c99f3b9b1735', '{"sim_uuid": "9

In [203]:
for network_event_dict in net_events_post_proc:
    if network_event_dict["start"] is not "" and network_event_dict["stop"] is not "":
        colors.append((random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)))
        df.append(dict(Task=network_event_dict["uuid_node"], Start=network_event_dict["start"], Finish=network_event_dict["stop"], Event_Name="connection"))

        

In [205]:
fig = ff.create_gantt(df, title="ITsim Network Events", colors=colors, index_col='Event_Name', showgrid_x=True, showgrid_y=True, show_colorbar=True, group_tasks=True)

iplot(fig)